In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd

In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-05/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di databse sumber.")

In [4]:
df.head(10)

In [5]:
# Input readable date
start_readable_date = "08-04-2025 00:00:00"  # Format: DD-MM-YYYY HH:MM:SS
end_readable_date = "08-04-2025 09:00:00"  # Format: DD-MM-YYYY HH:MM:SS

# Convert readable date to Unix timestamp
start_timestamp = int(datetime.strptime(start_readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(start_timestamp)
# end_timestamp = int(datetime.now().timestamp())  # Current time in Unix timestamp
end_timestamp = int(datetime.strptime(end_readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(end_timestamp)
# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

In [6]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [7]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-03/data', app=dest_app)

In [8]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>